In [ ]:
from glass.ng.sql.q import exec_write_q

conParam = {
    "HOST" : "localhost", "USER" : "postgres", "PORT" : "5432",
    "PASSWORD" : "admin", 'DATABASE' : 'viewtofire'
}

tbl_a = 'zip_vistoburn_1a12'
tbl_b = 'zip_vistoburn_d1b1a12'
tbl_o = 'zip_vistoburn_a'

q = (
    "CREATE TABLE {} AS "
    "SELECT "
    "CASE "
        "WHEN mtbl.rowi IS NOT NULL THEN "
        "mtbl.rowi ELSE jtbl.rowi "
    "END AS rowi, "
    "CASE "
        "WHEN mtbl.coli IS NOT NULL THEN "
        "mtbl.coli ELSE jtbl.coli "
    "END AS coli, "
    "CASE "
        "WHEN jtbl.pntid IS NULL THEN "
        "mtbl.pntid ELSE CASE "
            "WHEN mtbl.pntid IS NULL THEN "
            "jtbl.pntid ELSE mtbl.pntid || jtbl.pntid "
        "END "
    "END AS pntid "
    "FROM {} AS mtbl "
    "FULL JOIN {} AS jtbl "
    "ON mtbl.rowi = jtbl.rowi AND mtbl.coli = jtbl.coli"
)

exec_write_q(conParam, q.format(tbl_o, tbl_a, tbl_b))

In [ ]:
"""
For every SQL File in folder:

Restore Database, run some queries and Dump result
"""

conParam = {
    "HOST" : "localhost", "USER" : "postgres", "PORT" : "5432",
    "PASSWORD" : "admin"
}

sql_fld = '/home/jasp/mrgis/dbs'
outfld = '/home/jasp/mrgis/zipdb'

QS = [
    # Create ZIP Table
    ("CREATE TABLE zip_vistoburn AS "
     "SELECT rowi, coli, array_agg(pntid) AS pntid "
     "FROM vistoburn GROUP BY rowi, coli"),
    # Delete vistoburn
    "DROP TABLE IF EXISTS vistoburn"
]

import os
from glass.ng.sql import psql_cmd
from glass.pys.oss import lst_ff, fprop
from glass.ng.sql.mng.tbl import exec_write_q
from glass.ng.sql.fm import dump_db
from glass.ng.sql.db import create_db, drop_db

sqls = lst_ff(sql_fld)

for sql in sqls:
    # Restore database
    conParam["DATABASE"] = create_db(conParam, fprop(sql, 'fn'))
    psql_cmd(conParam, sql)

    # Execute queries
    exec_write_q(conParam, QS)

    # Dump Database
    dump_db(conParam, os.path.join(outfld, os.path.basename(sql)), api='psql')
    db = conParam["DATABASE"]
    
    # Drop Database
    del conParam["DATABASE"]
    drop_db(conParam, db)